In [ ]:
import pandas as pd


In [ ]:
# Load relevant files (adjust path as needed)
patients = pd.read_csv("patients.csv")
conditions = pd.read_csv("conditions.csv")
medications = pd.read_csv("medications.csv")
observations = pd.read_csv("observations.csv")
encounters = pd.read_csv("encounters.csv")


In [ ]:
# Pick a sample patient
sample_patient_id = patients.iloc[0]['Id']
sample_patient = patients[patients['Id'] == sample_patient_id]

# Filter related data
patient_conditions = conditions[conditions['PATIENT'] == sample_patient_id]
patient_medications = medications[medications['PATIENT'] == sample_patient_id]
patient_observations = observations[observations['PATIENT'] == sample_patient_id]
patient_encounters = encounters[encounters['PATIENT'] == sample_patient_id]

# Let's preview
print("Patient Basic Info:\n", sample_patient)
print("\nConditions:\n", patient_conditions[['DESCRIPTION', 'START']])
print("\nMedications:\n", patient_medications[['DESCRIPTION', 'START']])
print("\nObservations:\n", patient_observations[['DESCRIPTION', 'VALUE', 'DATE']])


Patient Basic Info:
                                      Id   BIRTHDATE DEATHDATE          SSN  \
0  30a6452c-4297-a1ac-977a-6a23237c7b46  1994-02-06       NaN  999-52-8591   

     DRIVERS    PASSPORT PREFIX      FIRST   MIDDLE      LAST  ...       CITY  \
0  S99996852  X47758697X    Mr.  Joshua658  Alvin56  Kunde533  ...  Braintree   

           STATE          COUNTY     FIPS   ZIP        LAT        LON  \
0  Massachusetts  Norfolk County  25021.0  2184  42.211142 -71.045802   

  HEALTHCARE_EXPENSES HEALTHCARE_COVERAGE  INCOME  
0            56904.96            18019.99  100511  

[1 rows x 28 columns]

Conditions:
                                    DESCRIPTION       START
0             Housing unsatisfactory (finding)  2012-04-01
1          Received higher education (finding)  2012-04-01
2                  Transport problem (finding)  2012-04-01
3                   Social isolation (finding)  2012-04-01
4                             Stress (finding)  2012-04-01
5            Medi

In [ ]:
# Example prompt construction
prompt = f"""
You are a medical assistant. Summarize the medical history of a patient for them in simple language.

Patient conditions:
{patient_conditions['DESCRIPTION'].tolist()}

Medications prescribed:
{patient_medications['DESCRIPTION'].tolist()}

Recent observations:
{patient_observations[['DESCRIPTION', 'VALUE']].head(5).values.tolist()}

Please generate a friendly summary for the patient.
"""

print(prompt)



You are a medical assistant. Summarize the medical history of a patient for them in simple language.

Patient conditions:
['Housing unsatisfactory (finding)', 'Received higher education (finding)', 'Transport problem (finding)', 'Social isolation (finding)', 'Stress (finding)', 'Medication review due (situation)', 'Full-time employment (finding)', 'Lack of access to transportation (finding)', 'Fracture of bone (disorder)', 'Closed fracture of hip (disorder)', 'Limited social contact (finding)', 'Victim of intimate partner abuse (finding)', 'Gingivitis (disorder)', 'Medication review due (situation)', 'Stress (finding)', 'Sprain (morphologic abnormality)', 'Sprain of ankle (disorder)', 'Acute viral pharyngitis (disorder)', 'Medication review due (situation)', 'Prediabetes (finding)', 'Victim of intimate partner abuse (finding)', 'Gingivitis (disorder)']

Medications prescribed:
['Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet', 'Ibuprofen 200 MG Oral Tablet', 'Acetami

In [ ]:
prompt = f"""
You are a medical assistant. Summarize the medical history of a patient for them in simple language.

Example 1:
Conditions: Hypertension, Type 2 Diabetes, Asthma
Medications: Metformin, Lisinopril, Albuterol
Observations: Blood Pressure: 145/90, Blood Sugar: 150 mg/dL, BMI: 30

Patient Summary: The patient has a history of hypertension, Type 2 diabetes, and asthma. They are on medications to control their blood pressure, manage diabetes, and treat asthma. Their blood pressure and blood sugar levels are slightly elevated, and their BMI is in the overweight range.

Example 2:
Conditions: None
Medications: None
Observations: Blood Pressure: 120/80, Blood Sugar: 90 mg/dL, BMI: 22

Patient Summary: The patient has no significant medical conditions and is in good health. Their blood pressure, blood sugar, and BMI are all within normal ranges.

Patient conditions:
{patient_conditions['DESCRIPTION'].tolist()}

Medications prescribed:
{patient_medications['DESCRIPTION'].tolist()}

Recent observations:
{patient_observations[['DESCRIPTION', 'VALUE']].head(5).values.tolist()}

Please generate a friendly summary for the patient.
"""


In [ ]:
from transformers import pipeline

# Load pre-trained model for summarization
summarizer = pipeline("summarization")

# Your long medical text here
text = """
Your long medical history or patient data text goes here.
"""

# Generate summary
summary = summarizer(text, max_length=150, min_length=50, do_sample=False)
print(summary)


In [ ]:
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

prompt = "Patient has a history of stress, fractures, and prediabetes. Please generate a health summary."

response = generator(prompt, max_length=100)
print(response)


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

inputs = tokenizer("Patient has a history of stress, fractures, and prediabetes. Please generate a health summary", return_tensors='pt', max_length=1024, truncation=True)
summary_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)


In [ ]:
# Load the pre-trained BART model and tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

# Your long text input (for example, a patient's medical data or any other long document)
text = """
Your long medical history or patient data text goes here.
This could include all the symptoms, treatments, diagnosis, and more details about the patient's condition.
For example: The patient has a history of hypertension and diabetes and is currently being treated with medication X and Y.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt', max_length=1024, truncation=True)

# Generate a summary using the model
summary_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, early_stopping=True)

# Decode the generated summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print the generated summary
print("Summary:")
print(summary)


Summary:
Your long medical history or patient data text goes here. This could include all the symptoms, treatments, diagnosis, and more details about the patient's condition. For example: The patient has a history of hypertension and diabetes and is currently being treated with medication X and Y. For confidential support call the Samaritans on 08457 90 90 90, visit a local Samaritans branch or see www.samaritans.org.
